# Nested Sampling Algorithm

- Jeffrey's prior for $D_0$
- Uniform Priors for $\alpha$, $f$ and $\sigma^2_{mn}$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import scipy.stats as stats

In [2]:
def jeff(x,magmin, magmax):
    return magmin*exp(u*log(magmax/magmin))
def uni(x, magmin, magmax):
    return 1/(magmax, magmin)


In [3]:
@jit
def noisylikelihoodcalc(posn, DT, D, ALPHA, F, varmn):
    # time series
    n = len(posn)

    #init value
    logl = 0
    k_BT =1
    

    # Intial values of the variance mean both noisy and clean
    mob = D * (np.abs(posn[0]) ** ALPHA)
    std_dev_noise = 2 * mob * DT + 2 * varmn
    noisy_mean = posn[0] + (mob * F + k_BT * (ALPHA * D * ((np.abs(posn[0]) ** (ALPHA - 1)) * (np.sign(posn[0]))))) * DT
    
    # log likelihood at 1st step
    logl = logl - ((posn[1] - noisy_mean) ** 2) / (2 * std_dev_noise) - np.log(np.sqrt(2 * np.pi * std_dev_noise))
    
    for i in range(2,n):
        # Mobility at the step
        mob = D * np.abs(posn[i - 1]) ** ALPHA

        # clean variance at the jth step
        std_dev = 2 * mob * DT

        # clean mean position at the jth time step
        mean_dist = posn[i - 1] + (mob * F + ALPHA * D * (np.abs(posn[i - 1]) ** (ALPHA - 1)) * (np.sign(posn[i - 1]))) * DT

        # noisy mean position at the jth step
        noisy_mean = mean_dist - varmn/(std_dev_noise) * (posn[i-1] - noisy_mean)

        # noisy variance at the jth posn
        std_dev_noise = std_dev + varmn * (2 - varmn / std_dev_noise)

        # calculation of log likelihood
        logl = logl - ((posn[i] - noisy_mean) ** 2) / (2 * std_dev_noise) - 0.5 * np.log(2 * np.pi * std_dev_noise)
    
    return logl

In [4]:
@jit
def Evidence(data, theta, npoints, tol):
    logZ = 0
    walker = walkergen(npoints, theta_max, theta_min)
    for i in range(0,npoints):
        ll[i] = noisylikelihoodcalc(p_data, 1, walker[i][0], walker[i][1], walker[i][2], walker[i][3])
    
    

In [5]:
@jit
def walkergen(n, tmax, tmin):
    x = np.zeros((n,4))
    for i in range(0,n):
        for j in range(0,4):
            x[i][j] = np.random.uniform(tmin[j],tmax[j])
    return x

In [6]:
@jit 
def logsum(a, b):
    return max(a,b) + np.log(1+ np.exp(-np.abs(a-b)))

In [7]:
D_min = 10**(-4)
D_max = 10**2
# D_min = 0.
# D_max = 1.


f_min = -1.
f_max = 1.

alpha_min = -2.
alpha_max = 2.

varmn_min = 0.
varmn_max = 1.

theta_max = np.array([D_max, alpha_max, f_max, varmn_max])
theta_min = np.array([D_min, alpha_min, f_min, varmn_min])


In [8]:
npoints = 100

walkers = np.zeros((npoints,4))

walker = walkergen(npoints, theta_max, theta_min)


p_data = np.array([50,46.743,46.751,40.915,45.066,50.56,44.448,40.518,40.145,39.551,42.887,40.218,44.399,38.416,36.626,35.867,30.252,28.843,26.412,27.815,28.166,32.899,34.738,33.237,24.448,22.771,19.285,24.966,23.912,23.839,23.83,22.253,15.661,20.156,20.166,16.9,15.495,18.734,18.234,17.823,16.052,11.424,9.4223,7.3746,7.2549,7.2847,6.7366,7.7324,6.8572,7.034,5.3564,3.7969,3.4073,4.3653,4.56,4.8884,3.8021,2.9259,3.5701,4.0879,2.3471,0.95298,1.7231,1.7968,1.753,2.1651,2.1354,2.404,2.9188,1.1974,1.3358,1.428,1.8476,2.1991,1.1013,1.3227,0.71634,0.68376,0.89795,1.6167,2.0934,2.4119,1.0693,1.2777,2.2396,0.30283,0.64551,-0.44319,-0.79814,-0.82102,-2.0478,-1.6834,-1.8176,-1.3092,-1.2355,-1.8231,-1.8543,-2.1829,-4.2785,-4.2926])

ll = np.zeros(npoints)

In [9]:
# walkers = np.zeros((npoints, 4))
walkers = np.array(walkergen(npoints, theta_max,theta_min))
walkers.shape

(100, 4)

# MCMC walker

Tried my hand at coding a MCMC walker using the Metropolis Hastings algo. The proposal function is a standard gaussian.

In [12]:
# @jit
def MCMCwalker(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,4):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

# Nested Sampling Code starts here

In [10]:
Nsteps = 10000

tol = -4




wt = 1/(npoints + 1)

Z = np.exp(min(ll))*wt

npoints = 100

walkers = np.array(walkergen(npoints, theta_max,theta_min))

for i in range(0,npoints):
    ll[i] = noisylikelihoodcalc(p_data, 1, *walkers[i])

# wt = 1/(npoints + 1)
logwt = -np.log(npoints + 1)
logZrem = np.log(0)
logZ = np.log(0)
logp = min(ll)+ logwt
logZ = logsum(logZ,logp)
for i in range(0, npoints):
    logZrem = logsum(ll[i], logZrem)
logZrem = logZrem + logwt

logZ

/tmp/ipykernel_18128/1388972089.py:21: RuntimeWarning: divide by zero encountered in log
  logZrem = np.log(0)
/tmp/ipykernel_18128/1388972089.py:22: RuntimeWarning: divide by zero encountered in log
  logZ = np.log(0)


-423256.85768848605

In [13]:
step = 0

j=1
tol = np.log(10**(-4))
lstar = []

# for i in range(0,npoints):
    # logZrem = logsum(logZrem, )

R = logZrem - logZ

if R< tol:
    print(logZ)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    logl = [noisylikelihoodcalc(p_data, 1, *walkers[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstar.append(min(logl))

    p = np.argmin(logl)
    # print(p)
    
    # log Z finding
    logZ = logsum(logZ, (lstar[-1] + logwt))

    print("Evidence")
    print(logZ)
    
    logZrem = np.log(0)
    
    for i in range(0, npoints):
        logZrem = logsum(logl[i], logZrem)
        
    logZrem = logZrem + logwt

    print( "Zrem =  "+str(logZrem))
    print("lowest likelihood = "+str(min(logl)))

    # ratio that determines the stopping ratio
    R = logZrem - logZ
    print("ratio")
    print(R)
    if R < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers[newwalkernumber]

    # walker replacement  
    trace = MCMCwalker(p_data, 1, thetawalker, lstar[-1] , Nsteps, step)

    step = trace[0]

    # print(trace[1])
    walkers[p] = trace[1]

LogfinalZ = logsum(logZ, logZrem)
LogfinalZ

Evidence
-423255.7689935255
Zrem =  -333.96170165378606
lowest likelihood = -423252.2425679692
ratio
422921.8072918717
Evidence
-90514.53667646188
Zrem =  -321.91621457121283
lowest likelihood = -90509.89170495248
ratio
90192.62046189066
Evidence
-84736.11616989953
Zrem =  -300.45397032366776
lowest likelihood = -84731.46124805928
ratio
84435.66219957586
Evidence
-55700.51997612227
Zrem =  -297.2529057603371
lowest likelihood = -55695.85510395116
ratio
55403.26707036193


/tmp/ipykernel_18128/788653818.py:34: RuntimeWarning: divide by zero encountered in log
  logZrem = np.log(0)


Evidence
-50423.983811301885
Zrem =  -297.2628560870602
lowest likelihood = -50419.30898879992
ratio
50126.72095521483
Evidence
-43706.84547529852
Zrem =  -230.3277006973434
lowest likelihood = -43702.160702465706
ratio
43476.517774601176
Evidence
-40693.545301943246
Zrem =  -230.33765102819657
lowest likelihood = -40688.85057877958
ratio
40463.20765091505
Evidence
-35146.909807182994
Zrem =  -230.34760135904975
lowest likelihood = -35142.205133688476
ratio
34916.562205823946
Evidence
-34523.17035042922
Zrem =  -230.35755168990292
lowest likelihood = -34518.455726603854
ratio
34292.81279873932
Evidence
-30692.60164885012
Zrem =  -230.3675020207561
lowest likelihood = -30687.877074693894
ratio
30462.234146829363
Evidence
-28334.884101518466
Zrem =  -230.37745235160926
lowest likelihood = -28330.14957703139
ratio
28104.50664916686
Evidence
-28311.87518329351
Zrem =  -230.38740268246244
lowest likelihood = -28307.130708475677
ratio
28081.487780611045
Evidence
-22856.856177132584
Zrem =  -

-187.22903993659446

In [19]:
len(lstar)

1734

In [20]:
10**(-81.786)

1.6368165214278035e-82

In [15]:
np.exp(-187.229)

4.8694335955322697e-82

In [81]:
N=500000
a = walkergen(1,theta_max,theta_min)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
a,lla

(array([ 6.97170044,  1.88013819, -0.38935263,  0.87740753]),
 -6208.648723079805)

In [83]:
a = walkergen(1,theta_max,theta_min)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
print("init_Walker = " + str(a))
print("init likelihood = " +str(lla))
t = MCMCwalker(p_data, 1, a, lla, N, step)
step = t[0]
fp = t[2]
llf = noisylikelihoodcalc(p_data, 1, fp[0], fp[1], fp[2], fp[3])
fp
t[1]
llf,fp

init_Walker = [4.04643391 1.3696873  0.68423192 0.28069053]
init likelihood = -3145.2794342032416


(-179.1692711507324,
 array([ 0.14287875,  1.05337414, -0.27326816,  0.02651726]))

In [19]:
llt = noisylikelihoodcalc(p_data, 1, 0.2, 1,-0.3,0)
llt

-180.397045194095